In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pickle

In [4]:
def create_improved_model(image_x, image_y):
    """Create improved model with better architecture and regularization"""
    num_of_classes = 15
    model = Sequential()
    
    # First conv block with batch normalization
    model.add(Conv2D(32, (5, 5), input_shape=(image_x, image_y, 1), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
    
    # Second conv block
    model.add(Conv2D(64, (5, 5), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
    
    # Dense layers with reduced dropout
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.3))  # Reduced from 0.6
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.3))  # Reduced from 0.6
    model.add(Dense(num_of_classes, activation='softmax'))

    # Better optimizer and compilation
    model.compile(
        loss='categorical_crossentropy', 
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        metrics=['accuracy']
    )
    
    # Improved callbacks
    filepath = "model_trad/QuickDraw_improved.keras"
    callbacks_list = [
        ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, 
                       save_best_only=True, mode='max'),
        EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.0001)
    ]

    return model, callbacks_list

def create_data_augmentation():
    """Create data augmentation generator for domain robustness"""
    datagen = ImageDataGenerator(
        rotation_range=15,      # Random rotations ±15°
        width_shift_range=0.1,  # Random horizontal shifts
        height_shift_range=0.1, # Random vertical shifts
        zoom_range=0.1,         # Random zoom ±10%
        shear_range=0.1,        # Random shear transformations
        fill_mode='constant',   # Fill with black (0)
        cval=0
    )
    return datagen

def load_and_preprocess_data():
    """Load and preprocess data with better practices"""
    # Load data
    with open("../features_onTrad", "rb") as f:
        features = np.array(pickle.load(f))
    with open("../labels_onTrad", "rb") as f:
        labels = np.array(pickle.load(f))
    
    print(f"Loaded data shapes: {features.shape}, {labels.shape}")
    
    # Shuffle data
    features, labels = shuffle(features, labels, random_state=42)
    
    # Convert labels to categorical
    labels = tf.keras.utils.to_categorical(labels)
    
    # Split data with proper validation set
    train_x, temp_x, train_y, temp_y = train_test_split(
        features, labels, test_size=0.2, random_state=42, stratify=labels
    )
    val_x, test_x, val_y, test_y = train_test_split(
        temp_x, temp_y, test_size=0.5, random_state=42, stratify=temp_y
    )
    
    # Reshape for CNN
    train_x = train_x.reshape(train_x.shape[0], 28, 28, 1)
    val_x = val_x.reshape(val_x.shape[0], 28, 28, 1)
    test_x = test_x.reshape(test_x.shape[0], 28, 28, 1)
    
    print(f"Split sizes - Train: {len(train_x)}, Val: {len(val_x)}, Test: {len(test_x)}")
    
    return train_x, val_x, test_x, train_y, val_y, test_y

In [5]:
# Load and preprocess data
train_x, val_x, test_x, train_y, val_y, test_y = load_and_preprocess_data()
    
# Create model and callbacks
model, callbacks = create_improved_model(28, 28)
    
print("\n📊 Model Architecture:")
model.summary()

Loaded data shapes: (150000, 784), (150000, 1)
Split sizes - Train: 120000, Val: 15000, Test: 15000


c:\Users\ACER\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



📊 Model Architecture:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 24, 24, 32)     │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 24, 24, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 12, 12, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 8, 8, 64)       │        51,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 8, 8, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 4, 4, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 15)             │         1,935 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 644,879 (2.46 MB)

 Trainable params: 644,687 (2.46 MB)

 Non-trainable params: 192 (768.00 B)

In [6]:
# Create data augmentation
datagen = create_data_augmentation()
datagen.fit(train_x)

In [7]:
print(f"\n🏃 Starting training...")
history = model.fit(
        datagen.flow(train_x, train_y, batch_size=64),
        validation_data=(val_x, val_y),
        steps_per_epoch=len(train_x) // 64,
        epochs=20,  # Increased from 3
        callbacks=callbacks,
        verbose=1
    )


🏃 Starting training...
Epoch 1/20


c:\Users\ACER\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.6973 - loss: 1.0198
Epoch 1: val_accuracy improved from -inf to 0.89720, saving model to model_trad/QuickDraw_improved.keras
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 58s 30ms/step - accuracy: 0.6974 - loss: 1.0196 - val_accuracy: 0.8972 - val_loss: 0.3677 - learning_rate: 0.0010
Epoch 2/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.8743 - loss: 0.4435
Epoch 2: val_accuracy improved from 0.89720 to 0.91020, saving model to model_trad/QuickDraw_improved.keras
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 57s 31ms/step - accuracy: 0.8743 - loss: 0.4435 - val_accuracy: 0.9102 - val_loss: 0.3227 - learning_rate: 0.0010
Epoch 3/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.8969 - loss: 0.3702
Epoch 3: val_accuracy improved from 0.91020 to 0.92687, saving model to model_trad/QuickDraw_improved.keras
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 48s 26ms/step - accuracy: 0.8969 - loss: 0.3702 - val_accuracy: 0.9269 - val_loss: 0.2722 - learning_r

In [8]:
# Evaluate on test set
print(f"\n📊 Final Evaluation:")
test_loss, test_acc = model.evaluate(test_x, test_y, verbose=0)
print(f"   Test Accuracy: {test_acc:.4f}")
print(f"   Test Loss: {test_loss:.4f}")


📊 Final Evaluation:
   Test Accuracy: 0.9520
   Test Loss: 0.1823


In [9]:
model.save('model_trad/QuickDraw_improved_final.keras')
print(f"   ✅ Model saved as 'QuickDraw_improved_final.keras'")

   ✅ Model saved as 'QuickDraw_improved_final.keras'
